In [12]:
from snowflake.snowpark import Session
import time

In [13]:
def get_snowflake_session():
    """Create a Snowflake session using ACCOUNTADMIN credentials."""
    try:
        from typing import Dict, Union

        connection_parameters: Dict[str, Union[str, int]] = {
            "account": "FVQCWWK-IJB71419",
            "user": "LALIT",
            "password": "Klalitkumar@2025",
            "authenticator": "snowflake",
            "role": "ACCOUNTADMIN",
            "warehouse": "COMPUTE_WH",
            "database": "SNOWFLAKE",
            "schema": "PUBLIC"
        }
        session = Session.builder.configs(connection_parameters).create()
        print("Snowflake session established successfully.")
        return session
    except Exception as e:
        print(f"Error establishing Snowflake session: {str(e)}")
        raise

In [14]:
def check_and_create_role(session):
    """Check if role exists and create if not present."""
    start_time = time.time()
    role_name = "RECOMMENDATION_SYSTEM_ROLE"
    try:
        # Check if role exists
        roles_df = session.sql(f"""
            SELECT NAME FROM SNOWFLAKE.ACCOUNT_USAGE.ROLES
            WHERE NAME = '{role_name}' AND DELETED IS NULL
        """).to_pandas()
        if not roles_df.empty:
            print(f"Role {role_name} already exists.")
        else:
            session.sql(f"""
                CREATE ROLE IF NOT EXISTS {role_name}
                COMMENT = 'Role for Dealer Product Recommendation System'
            """).collect()
            print(f"Created role: {role_name}")

        # Grant role to ACCOUNTADMIN and user
        session.sql(f"""
            GRANT ROLE {role_name} TO ROLE ACCOUNTADMIN
        """).collect()
        print(f"Granted {role_name} to ACCOUNTADMIN")
        session.sql(f"""
            GRANT ROLE {role_name} TO USER LALIT
        """).collect()
        print(f"Granted {role_name} to user LALIT")

        print(f"check_and_create_role completed in {time.time() - start_time:.2f} seconds")
        return role_name
    except Exception as e:
        print(f"Error in check_and_create_role: {str(e)}")
        raise

In [15]:
def check_and_create_warehouse(session):
    """Check if warehouse exists and create if not present."""
    start_time = time.time()
    warehouse_name = "RECOMMENDATION_SYSTEM_WH"
    role_name = "RECOMMENDATION_SYSTEM_ROLE"
    try:
        # Check if warehouse exists
        warehouses_df = session.sql(f"""
            SELECT WAREHOUSE_NAME FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSES
            WHERE WAREHOUSE_NAME = '{warehouse_name}' AND DELETED IS NULL
        """).to_pandas()
        if not warehouses_df.empty:
            print(f"Warehouse {warehouse_name} already exists.")
        else:
            session.sql(f"""
                CREATE WAREHOUSE IF NOT EXISTS {warehouse_name}
                WITH
                    WAREHOUSE_SIZE = 'XSMALL'
                    AUTO_SUSPEND = 300
                    AUTO_RESUME = TRUE
                    COMMENT = 'Warehouse for Dealer Product Recommendation System'
            """).collect()
            print(f"Created warehouse: {warehouse_name}")

        # Grant warehouse privileges
        session.sql(f"""
            GRANT ALL ON WAREHOUSE {warehouse_name} TO ROLE {role_name}
        """).collect()
        print(f"Granted ALL privileges on warehouse {warehouse_name} to {role_name}")

        print(f"check_and_create_warehouse completed in {time.time() - start_time:.2f} seconds")
        return warehouse_name
    except Exception as e:
        print(f"Error in check_and_create_warehouse: {str(e)}")
        raise

In [16]:
def check_and_create_database(session):
    """Check if database exists and create if not present."""
    start_time = time.time()
    database_name = "RECOMMENDATION_SYSTEM_DB"
    role_name = "RECOMMENDATION_SYSTEM_ROLE"
    try:
        # Check if database exists
        databases_df = session.sql(f"""
            SELECT DATABASE_NAME FROM SNOWFLAKE.ACCOUNT_USAGE.DATABASES
            WHERE DATABASE_NAME = '{database_name}' AND DELETED IS NULL
        """).to_pandas()
        if not databases_df.empty:
            print(f"Database {database_name} already exists.")
        else:
            session.sql(f"""
                CREATE DATABASE IF NOT EXISTS {database_name}
                COMMENT = 'Database for Dealer Product Recommendation System'
            """).collect()
            print(f"Created database: {database_name}")

        # Grant database privileges
        session.sql(f"""
            GRANT ALL ON DATABASE {database_name} TO ROLE {role_name}
        """).collect()
        print(f"Granted ALL privileges on database {database_name} to {role_name}")

        print(f"check_and_create_database completed in {time.time() - start_time:.2f} seconds")
        return database_name
    except Exception as e:
        print(f"Error in check_and_create_database: {str(e)}")
        raise

In [17]:
def check_and_create_schemas(session, database_name):
    """Check if schemas exist and create if not present."""
    start_time = time.time()
    schemas = ["ODS", "STG", "DWH"]
    role_name = "RECOMMENDATION_SYSTEM_ROLE"
    try:
        # Check existing schemas
        schemas_df = session.sql(f"""
            SELECT SCHEMA_NAME FROM {database_name}.INFORMATION_SCHEMA.SCHEMATA
            WHERE SCHEMA_NAME IN ('ODS', 'STG', 'DWH')
        """).to_pandas()
        existing_schemas = schemas_df['SCHEMA_NAME'].tolist()
        print(f"Existing schemas in {database_name}: {existing_schemas}")

        # Create missing schemas
        for schema in schemas:
            if schema not in existing_schemas:
                session.sql(f"""
                    CREATE SCHEMA IF NOT EXISTS {database_name}.{schema}
                    COMMENT = '{schema} schema for Dealer Product Recommendation System'
                """).collect()
                print(f"Created schema: {database_name}.{schema}")
            else:
                print(f"Schema {database_name}.{schema} already exists.")

            # Grant schema privileges
            session.sql(f"""
                GRANT ALL ON SCHEMA {database_name}.{schema} TO ROLE {role_name}
            """).collect()
            print(f"Granted ALL privileges on schema {database_name}.{schema} to {role_name}")

        print(f"check_and_create_schemas completed in {time.time() - start_time:.2f} seconds")
        return schemas
    except Exception as e:
        print(f"Error in check_and_create_schemas: {str(e)}")
        raise

In [18]:
def check_and_create_tables(session, database_name):
    """Check if tables exist and create if not present."""
    start_time = time.time()
    role_name = "RECOMMENDATION_SYSTEM_ROLE"
    tables = ["DEALERS", "PRODUCTS", "SALES", "DP_MAPPING", "EC_CLUB"]
    try:
        # Check existing tables
        tables_df = session.sql(f"""
            SELECT TABLE_NAME 
            FROM {database_name}.INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_SCHEMA = 'ODS' 
            AND TABLE_NAME IN ('DEALERS', 'PRODUCTS', 'SALES', 'DP_MAPPING', 'EC_CLUB')
        """).to_pandas()
        existing_tables = tables_df['TABLE_NAME'].tolist()
        print(f"Existing tables in {database_name}.ODS: {existing_tables}")

        # Create missing tables
        if 'DEALERS' not in existing_tables:
            session.sql(f"""
                CREATE TABLE IF NOT EXISTS {database_name}.ODS.DEALERS (
                    DEALER_NO VARCHAR(50) PRIMARY KEY,
                    DEALER_NAME VARCHAR(100),
                    REGION VARCHAR(50),
                    ADDRESS VARCHAR(200),
                    ANNUAL_REVENUE DECIMAL(18,2),
                    DEALER_TYPE VARCHAR(20),
                    EC_CLUB INTEGER,
                    TOTAL_SALES DECIMAL(18,2) DEFAULT 0,
                    TOTAL_INVOICES INTEGER DEFAULT 0,
                    AVERAGE_BILL_VALUE DECIMAL(18,2) DEFAULT 0,
                    AVERAGE_SKUS_PER_INVOICE DECIMAL(18,2) DEFAULT 0
                )
                COMMENT = 'Table storing dealer information'
            """).collect()
            print(f"Created table: {database_name}.ODS.DEALERS")
        else:
            print(f"Table {database_name}.ODS.DEALERS already exists.")

        if 'PRODUCTS' not in existing_tables:
            session.sql(f"""
                CREATE TABLE IF NOT EXISTS {database_name}.ODS.PRODUCTS (
                    SKU VARCHAR(50) PRIMARY KEY,
                    PRODUCT_NAME VARCHAR(100),
                    PROD_CATEGORY VARCHAR(50),
                    PROD_RANGE VARCHAR(50),
                    PROD_SUBCATEGORY VARCHAR(50),
                    IS_TRADING INTEGER,
                    IS_FINISHED INTEGER
                )
                COMMENT = 'Table storing product information'
            """).collect()
            print(f"Created table: {database_name}.ODS.PRODUCTS")
        else:
            print(f"Table {database_name}.ODS.PRODUCTS already exists.")

        if 'SALES' not in existing_tables:
            session.sql(f"""
                CREATE TABLE IF NOT EXISTS {database_name}.ODS.SALES (
                    DEALER_NO VARCHAR(50),
                    SKU VARCHAR(50),
                    DATE DATE,
                    QUANTITY INTEGER,
                    AMOUNT DECIMAL(18,2),
                    PRIMARY KEY (DEALER_NO, SKU, DATE)
                )
                COMMENT = 'Table storing sales transactions'
            """).collect()
            print(f"Created table: {database_name}.ODS.SALES")
        else:
            print(f"Table {database_name}.ODS.SALES already exists.")

        if 'DP_MAPPING' not in existing_tables:
            session.sql(f"""
                CREATE TABLE IF NOT EXISTS {database_name}.ODS.DP_MAPPING (
                    DN_NUMBER VARCHAR(50) PRIMARY KEY,
                    VERTICAL VARCHAR(50)
                )
                COMMENT = 'Table storing dealer-vertical mappings'
            """).collect()
            print(f"Created table: {database_name}.ODS.DP_MAPPING")
        else:
            print(f"Table {database_name}.ODS.DP_MAPPING already exists.")

        if 'EC_CLUB' not in existing_tables:
            session.sql(f"""
                CREATE TABLE IF NOT EXISTS {database_name}.ODS.EC_CLUB (
                    DEALER_NO VARCHAR(50) PRIMARY KEY,
                    EC_CLUB INTEGER
                )
                COMMENT = 'Table storing elite club membership'
            """).collect()
            print(f"Created table: {database_name}.ODS.EC_CLUB")
        else:
            print(f"Table {database_name}.ODS.EC_CLUB already exists.")

        # Grant table privileges
        for table in tables:
            session.sql(f"""
                GRANT ALL ON TABLE {database_name}.ODS.{table} TO ROLE {role_name}
            """).collect()
            print(f"Granted ALL privileges on table {database_name}.ODS.{table} to {role_name}")

        print(f"check_and_create_tables completed in {time.time() - start_time:.2f} seconds")
    except Exception as e:
        print(f"Error in check_and_create_tables: {str(e)}")
        raise

In [19]:
def grant_usage_privileges(session, warehouse_name, database_name):
    """Grant usage privileges to the role for warehouse and database."""
    start_time = time.time()
    role_name = "RECOMMENDATION_SYSTEM_ROLE"
    try:
        # Grant warehouse usage
        session.sql(f"""
            GRANT USAGE ON WAREHOUSE {warehouse_name} TO ROLE {role_name}
        """).collect()
        print(f"Granted USAGE on warehouse {warehouse_name} to {role_name}")

        # Grant database usage
        session.sql(f"""
            GRANT USAGE ON DATABASE {database_name} TO ROLE {role_name}
        """).collect()
        print(f"Granted USAGE on database {database_name} to {role_name}")

        print(f"grant_usage_privileges completed in {time.time() - start_time:.2f} seconds")
    except Exception as e:
        print(f"Error in grant_usage_privileges: {str(e)}")
        raise

In [20]:
def main():
    """Main function to set up the Snowflake environment with existence checks."""
    start_time = time.time()
    try:
        print("Starting Snowflake environment setup...")

        # Establish Snowflake session
        session = get_snowflake_session()

        # Check and create role
        role_name = check_and_create_role(session)

        # Check and create warehouse
        warehouse_name = check_and_create_warehouse(session)

        # Check and create database
        database_name = check_and_create_database(session)

        # Check and create schemas
        schemas = check_and_create_schemas(session, database_name)

        # Check and create tables
        check_and_create_tables(session, database_name)

        # Grant usage privileges
        grant_usage_privileges(session, warehouse_name, database_name)

        # Verify setup
        tables_df = session.sql(f"""
            SELECT TABLE_NAME 
            FROM {database_name}.INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_SCHEMA = 'ODS' 
            AND TABLE_NAME IN ('DEALERS', 'PRODUCTS', 'SALES', 'DP_MAPPING', 'EC_CLUB')
        """).to_pandas()
        tables_exist = tables_df['TABLE_NAME'].tolist()
        print(f"Tables in {database_name}.ODS: {tables_exist}")

        if len(tables_exist) == 5:
            print("All required tables verified successfully.")
        else:
            raise Exception(f"Expected 5 tables, but found {len(tables_exist)}: {tables_exist}")

        print(f"Snowflake environment setup completed in {time.time() - start_time:.2f} seconds")
    except Exception as e:
        print(f"Environment setup failed: {str(e)}")
        raise
    finally:
        session.close()
        print("Snowflake session closed.")

In [21]:
if __name__ == "__main__":
    main()

Starting Snowflake environment setup...
Snowflake session established successfully.
Error in check_and_create_role: (1304): 01bd535f-0305-ca7f-000a-ebe7000fb016: 000904 (42000): SQL compilation error: error line 2 at position 58
invalid identifier 'DELETED'
There are existing quoted column identifiers: []. Please use one of them to reference the column. See more details on Snowflake identifier requirements https://docs.snowflake.com/en/sql-reference/identifiers-syntax
Environment setup failed: (1304): 01bd535f-0305-ca7f-000a-ebe7000fb016: 000904 (42000): SQL compilation error: error line 2 at position 58
invalid identifier 'DELETED'
There are existing quoted column identifiers: []. Please use one of them to reference the column. See more details on Snowflake identifier requirements https://docs.snowflake.com/en/sql-reference/identifiers-syntax
Snowflake session closed.


SnowparkSQLException: (1304): 01bd535f-0305-ca7f-000a-ebe7000fb016: 000904 (42000): SQL compilation error: error line 2 at position 58
invalid identifier 'DELETED'
There are existing quoted column identifiers: []. Please use one of them to reference the column. See more details on Snowflake identifier requirements https://docs.snowflake.com/en/sql-reference/identifiers-syntax